In [1]:
import higlass
from higlass.client import View, Track, CombinedTrack
from higlass.tilesets import cooler, beddb, chromsizes, bigwig, Tileset
from higlass.viewer import save_b64_image_to_png
import clodius
import os
import os.path as path
import pandas as pd
import numpy as np
import itertools
import negspy.coordinates as nc

def bed2ddb(filepath, uuid=None, **kwargs):
    from clodius.tiles.bed2ddb import get_2d_tileset_info, get_2D_tiles
    from clodius.tiles.utils import tiles_wrapper_2d

    return Tileset(
        uuid=uuid,
        tileset_info=lambda: get_2d_tileset_info(filepath),
        tiles=lambda tids: tiles_wrapper_2d(
            tids, lambda z, x, y: get_2D_tiles(filepath, z, x, y)[(x, y)]
        ),
        **kwargs
    )


# Annotations

In [2]:
annot_dir = path.join("..", "Results", "2019-10-24_higlass")
genes = beddb(path.join(annot_dir, "Data", "hg38", "gene-annotations-hg38.beddb"))
chrom_sizes = chromsizes(path.join(annot_dir, "hg38.chrom.sizes"))
label_font_size = 18
chr_label_size = 30
annots_size = 150

chrom_labels = {
    p: Track(
        track_type=l + "-chromosome-labels",
        tileset=chrom_sizes,
        position=p,
        height=chr_label_size,
        width=chr_label_size,
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    ) for p, l in zip(["top", "bottom", "left", "right"], ["horizontal", "horizontal", "vertical", "vertical"])
}
chrom_grid = {
    p: Track(
        track_type=l + "-chromosome-grid",
        tileset=chrom_sizes,
        position=p,
        height=chr_label_size,
        width=chr_label_size,
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    ) for p, l in zip(
        ["top", "bottom", "left", "right"],
        ["horizontal", "horizontal", "vertical", "vertical"])
}

gene_annots = {
    p: Track(
        track_type=l + "-gene-annotations",
        tileset=genes,
        position=p,
        height=annots_size,
        width=annots_size,
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    ) for p, l in zip(["top", "bottom", "left", "right"], ["horizontal", "horizontal", "vertical", "vertical"])
}

hg38 = nc.get_chrominfo("hg38")

# Load data

## Metadata

In [3]:
meta = pd.read_csv("config.tsv", sep="\t")
meta = meta.loc[meta.Include == "Yes", :]

SAMPLES = {
    "all": meta["Sample_ID"].tolist(),
    "tumour": meta.loc[(meta.Source == "Primary") & (meta.Type == "Malignant"), "Sample_ID"].tolist(),
    "benign": meta.loc[(meta.Source == "Primary") & (meta.Type == "Benign"), "Sample_ID"].tolist(),
    "primary": meta.loc[(meta.Source == "Primary"), "Sample_ID"].tolist(),
    "clines": meta.loc[(meta.Source == "Cell Line"), "Sample_ID"].tolist(),
}
meta

,Sample_ID,Label,Include,Source,Type,Sample_Colour,Type_Colour,Tissue,T2E
0,PCa13266,CPCG0268,Yes,Primary,Malignant,#7F3C8D,#1F77B4,Prostate,No
1,PCa13848,CPCG0366,Yes,Primary,Malignant,#11A579,#1F77B4,Prostate,Yes
2,PCa14121,CPCG0255,Yes,Primary,Malignant,#3969AC,#1F77B4,Prostate,No
3,PCa19121,CPCG0258,Yes,Primary,Malignant,#F2B701,#1F77B4,Prostate,Yes
4,PCa3023,CPCG0324,Yes,Primary,Malignant,#E73F74,#1F77B4,Prostate,Yes
5,PCa33173,CPCG0246,Yes,Primary,Malignant,#80BA5A,#1F77B4,Prostate,No
6,PCa40507,CPCG0342,Yes,Primary,Malignant,#E68310,#1F77B4,Prostate,Yes
7,PCa51852,CPCG0336,Yes,Primary,Malignant,#008695,#1F77B4,Prostate,Yes
8,PCa53687,CPCG0339,Yes,Primary,Malignant,#CF1C90,#1F77B4,Prostate,No
9,PCa56413,CPCG0331,Yes,Primary,Malignant,#F97B72,#1F77B4,Prostate,Yes


## Contact Matrices

In [4]:
resolutions = [
    1000, 2000, 3000, 4000, 5000,
    10000, 20000, 30000, 40000, 50000,
    100000, 200000, 300000, 400000, 500000,
    1000000, 2000000, 3000000, 4000000, 5000000
][::-1]
min_resolution = 40000
colour_range = [
    "rgba(65, 105, 225, 1.0)",
    "rgba(255, 250, 250, 1.0)",
    "rgba(255, 25, 25, 1.0)"
]

mtx_dir = {
    "primary": path.join("..", "Data", "Processed", "2019-06-18_PCa-LowC-sequencing", "Contacts"),
    "clines": path.join("..", "Data", "External", "Rhie_2019", "Contacts"),
}
mtx_files = {}
for s in SAMPLES["primary"]:
    mtx_files[s] = path.join(mtx_dir["primary"], s + ".mcool")

for s in SAMPLES["clines"]:
    mtx_files[s] = path.join(mtx_dir["clines"], s + ".mcool")

mtx_tilesets = {
    s: cooler(f)
    for s, f in mtx_files.items()
}

contacts = {
    s: {
        p: Track(
            track_type=pl + "heatmap",
            datatype="matrix",
            filetype="cooler",
            tileset=mtx_tilesets[s],
            height=height,
            options={
                "maxZoom": str(resolutions.index(min_resolution)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
                "name": meta.loc[meta["Sample_ID"] == s, "Label"].values[0],
                "colorRange": colour_range,
            }
        )
        for p, pl, height in zip(
            ["top", "right", "bottom", "left", "center"],
            ["horizontal-", "horizontal-", "horizontal-", "horizontal-", ""],
            #["linear-", "linear-", "linear-", "linear-", ""],
            [240, 240, 240, 240, 500],
        )
    }
    for s in mtx_files.keys()
}

## Chromatin Interactions

In [5]:
loop_track_dir = path.join("..", "Results", "2020-10-06_loops", "Tracks")
loop_files = {
    "1D": {
        s: path.join(loop_track_dir, s + ".anchors.beddb")
        for s in SAMPLES["primary"]
    },
    "2D": {
        s: path.join(loop_track_dir, s + ".loops.bed2ddb")
        for s in SAMPLES["primary"]
    },
}
loop_tilesets = {
    "1D": {
        s: beddb(f)
        for s, f in loop_files["1D"].items()
    },
    "2D": {
        s: bed2ddb(f)
        for s, f in loop_files["2D"].items()
    },
}
loops = {
    "1D": {
        s: {
            p: Track(
                track_type="bedlike",
                datatype="bedlike",
                filetype="beddb",
                tileset=loop_tilesets["1D"][s],
                options={
                    "showMousePosition": True,
                    "name": meta.loc[meta["Sample_ID"] == s, "Label"].values[0] + " Interactions",
                    "lineStrokeColor": meta.loc[meta["Sample_ID"] == s, "Sample_Colour"].values[0],
                }
            )
            for p in ["top", "left", "bottom", "right"]
        }
        for s in SAMPLES["primary"]
    },
    "2D": {
        s: {
            p: Track(
                track_type="2d-rectangle-domains",
                datatype="2d-rectangle-domains",
                filetype="bed2ddb",
                tileset=loop_tilesets["2D"][s],
                height=height,
                options={
                    "showMousePosition": True,
                    "name": meta.loc[meta["Sample_ID"] == s, "Label"].values[0] + " Interactions",
                    "lineStrokeColor": meta.loc[meta["Sample_ID"] == s, "Sample_Colour"].values[0],
                    "rectangleDomainStrokeColor": meta.loc[meta["Sample_ID"] == s, "Sample_Colour"].values[0],
                    "rectangleDomainFillColor": meta.loc[meta["Sample_ID"] == s, "Sample_Colour"].values[0],
                    "rectangleDomainOpcaity": 0.4,
                }
            )
            for p, height in zip(
                ["top", "left", "bottom", "right", "center"],
                [50, 50, 50, 50, 600],
            )
        }
        for s in SAMPLES["primary"]
    },
}

## Compartmentalization

In [6]:
cmpt_files = {
    s: path.join("..", "Results", "2021-03-03_compartments", "Compartments", s + ".compartments.cis.bw")
    for s in SAMPLES["all"]
}
cmpt_tilesets = {s: bigwig(f) for s, f in cmpt_files.items()}
compartments = {
    s: {
        p: Track(
            track_type=t + "-line",
            filetype="vector",
            position=p,
            tileset=ts,
            height=100,
            options={
                "showMousePosition": True,
                "name": meta.loc[meta["Sample_ID"] == s, "Label"].values[0] + " Compartment",
                "lineStrokeColor": meta.loc[meta["Sample_ID"] == s, "Sample_Colour"].values[0],
                "valueScaleMin": -2,
                "valueScaleMax": 2,
                "zeroLineVisible": True,
                "zeroLineColor": "#000000",
            },
        ) for p, t in zip(
            ["top", "bottom", "left", "right"],
            ["horizontal", "horizontal", "vertical", "vertical"]
        )
    } for s, ts in cmpt_tilesets.items()
}

# Figures

## AR TADs

In [6]:
prostate_samples = (
    SAMPLES["primary"]
    + [
        "SRR7702334",
        "SRR7702335",
        "SRR8446383",
        "SRR8446384",
        "SRR8446385",
        "SRR8446386",
        "SRR8446387",
    ]
)

views = [View(
    tracks=[
        chrom_labels["top"],
        gene_annots["top"],
    ] + [
        contacts[s]["top"] for s in prostate_samples
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chrX", 65715000, hg38),
        nc.chr_pos_to_genome_pos("chrX", 69570000, hg38)
    ],
)]

d, s, v = higlass.display(
    views=views,
    zoom_syncs=[views],
    location_syncs=[views],
    server_port=8192,
)
d

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'CloUl0RGQsufSLS_MEvkVA', 'tracks': {'top': [{'ty…

## FOXA1 Chromatin Interactions

In [12]:
views = [View(
    tracks=[
        chrom_labels["top"],
        gene_annots["top"],
        chrom_labels["left"],
        gene_annots["left"],
        CombinedTrack(
            [ contacts["PCa13266"]["center"] ]
            + [ loops["2D"][s]["center"] for s in SAMPLES["primary"] ]
        ),
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 37100000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 37900000, hg38)
    ],
)]

d, s, v = higlass.display(
    views=views,
    zoom_syncs=[views],
    location_syncs=[views],
    server_port=8193,
)
d

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'Cl8-rL78Qv-nX4uIXXatDg', 'tracks': {'top': [{'ty…

## Compartmental Differences on chr19

In [8]:
views = [
    View(
        tracks=[
            chrom_labels["top"],
            gene_annots["top"],
            CombinedTrack([
                compartments[s]["top"]
                for s in SAMPLES["benign"]
            ]),
            CombinedTrack([
                compartments[s]["top"]
                for s in SAMPLES["tumour"]
            ]),
        ],
        initialXDomain=[
            nc.chr_pos_to_genome_pos("chr19", 0, hg38),
            nc.chr_pos_to_genome_pos("chr20", 0, hg38)
        ],
    ),
]

d, s, v = higlass.display(
    views=views,
    zoom_syncs=[views],
    location_syncs=[views],
    server_port=8195,
)
d

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'JyOUPt7rRq2M0YeQr6yx9A', 'tracks': {'top': [{'ty…

In [7]:
views = [
    View(
        tracks=[
            chrom_labels["top"],
            gene_annots["top"],
            CombinedTrack([
                compartments[s]["top"]
                for s in SAMPLES["benign"]
            ]),
            CombinedTrack([
                compartments[s]["top"]
                for s in ["PCa13266", "PCa13848", "PCa3023", "PCa33173", "PCa40507", "PCa53687", "PCa56413", "PCa58215"]
            ]),
            CombinedTrack([
                compartments[s]["top"]
                for s in ["PCa14121", "PCa19121", "PCa51852", "PCa57294"]
            ]),
        ],
        initialXDomain=[
            nc.chr_pos_to_genome_pos("chr19", 0, hg38),
            nc.chr_pos_to_genome_pos("chr20", 0, hg38)
        ],
    ),
]

d, s, v = higlass.display(
    views=views,
    zoom_syncs=[views],
    location_syncs=[views],
    server_port=8196,
)
d

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'IDhO-3giQOW330abdvd54Q', 'tracks': {'top': [{'ty…

In [8]:
views = [
    View(
        tracks=[
            chrom_labels["top"],
            gene_annots["top"],
            chrom_labels["left"],
            gene_annots["left"],
            contacts["Benign-Prostate-1595983"]["center"],
            compartments["Benign-Prostate-1595983"]["bottom"],
        ],
        initialXDomain=[
            nc.chr_pos_to_genome_pos("chr18", 63000000, hg38),
            nc.chr_pos_to_genome_pos("chr20", 21000000, hg38)
        ],
    ),
    View(
        tracks=[
            compartments["PCa19121"]["top"],
            chrom_labels["left"],
            gene_annots["left"],
            contacts["PCa19121"]["center"],
            chrom_labels["bottom"],
            gene_annots["bottom"],
        ],
        initialXDomain=[
            nc.chr_pos_to_genome_pos("chr18", 63000000, hg38),
            nc.chr_pos_to_genome_pos("chr20", 21000000, hg38)
        ],
    ),
]

d, s, v = higlass.display(
    views=views,
    zoom_syncs=[views],
    location_syncs=[views],
    #value_scale_syncs=[
    #    [
    #        (views[0], contacts["Benign-Prostate-1595983"]["center"]),
    #        (views[1], contacts["PCa19121"]["center"]),
    #    ]
    #],
    server_port=8194,
)
d

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'QxFdIs8GScSHO5fTiScWqg', 'tracks': {'top': [{'ty…

## Compartmental differences on chrY

In [9]:
views = [
    View(
        tracks=[
            chrom_labels["top"],
            gene_annots["top"],
            CombinedTrack([
                compartments[s]["top"]
                for s in SAMPLES["benign"]
            ]),
            CombinedTrack([
                compartments[s]["top"]
                for s in SAMPLES["tumour"]
            ]),
        ],
        initialXDomain=[
            nc.chr_pos_to_genome_pos("chrY", 0, hg38),
            nc.chr_pos_to_genome_pos("chrY", 22000000, hg38)
        ],
    ),
]

d, s, v = higlass.display(
    views=views,
    zoom_syncs=[views],
    location_syncs=[views],
    server_port=8198,
)
d

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'f3Mll1PDQyKMtOWHiPt8Ug', 'tracks': {'top': [{'ty…

In [16]:
views = [
    View(
        tracks=[
            chrom_labels["top"],
            gene_annots["top"],
            CombinedTrack([
                compartments[s]["top"]
                for s in SAMPLES["benign"]
            ]),
            CombinedTrack([
                compartments[s]["top"]
                for s in ["PCa40507", "PCa53687", "PCa57294", "PCa58215"]
            ]),
            CombinedTrack([
                compartments[s]["top"]
                for s in ["PCa13266", "PCa13848", "PCa14121", "PCa19121", "PCa3023", "PCa33173", "PCa51852", "PCa56413"]
            ]),
        ],
        initialXDomain=[
            nc.chr_pos_to_genome_pos("chrY", 0, hg38),
            nc.chr_pos_to_genome_pos("chrY", 22000000, hg38)
        ],
    ),
]

d, s, v = higlass.display(
    views=views,
    zoom_syncs=[views],
    location_syncs=[views],
    server_port=8196,
)
d

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'f-EIgfwOQfifMpLbugXYAg', 'tracks': {'top': [{'ty…

## Compartmental differences on chr3

In [9]:
views = [
    View(
        tracks=[
            chrom_labels["top"],
            gene_annots["top"],
            CombinedTrack([
                compartments[s]["top"]
                for s in SAMPLES["benign"]
            ]),
            CombinedTrack([
                compartments[s]["top"]
                for s in SAMPLES["tumour"]
            ]),
        ],
        initialXDomain=[
            nc.chr_pos_to_genome_pos("chr3", 0, hg38),
            nc.chr_pos_to_genome_pos("chr4", 0, hg38)
        ],
    ),
]

d, s, v = higlass.display(
    views=views,
    zoom_syncs=[views],
    location_syncs=[views],
    server_port=8198,
)
d

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'fWlKaFLGTe-sJiYxLEg-Uw', 'tracks': {'top': [{'ty…